![title](evema-logo.jpeg)

-Instructor: M.Sc.Saul Calderon.

- Autores: 
    - Saúl Calderón, Mauro Méndez, Manuel Zumbado. 

# Redes Convolucionales

**Importado de  Librerías**
Se importarán las librerías:


1.   Torch: Pytorch, librería para manipulación de matrices, grafos computacionales y redes neuronales
2.   Numpy: Librería para manipulación de matrices
3.   Pandas: Librería para manipular fuentes de datos, usada para escribir achivos .csv
4.  Torchvision para manejo de conjuntos de datos y transformadas en redes de pytorch





In [2]:
#ver este ejemplo de aca 
#https://medium.com/predict/using-pytorch-for-kaggles-famous-dogs-vs-cats-challenge-part-1-preprocessing-and-training-407017e1a10c

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Parma/LibroVersionMasReciente/4_Clasificacion/RedesNeuronales/SrcRedesConvolucionales"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CNN_CatsDogsResnet.ipynb  dataCatsDogs	     Prueba1.csv
CNN_CatsDogsSimple.ipynb  dogsCats_weights1  PruebaBorrosidad.csv
CNN_MNIST.ipynb		  evema-logo.jpeg    pruebas.ipynb
custom_datasets.py	  imgs
data			  mnist_weights1


In [0]:
from __future__ import print_function
import argparse
#Pytorch library import
import torch
import torch.nn as nn




import torch.nn.functional as F
import torch.optim as optim
import numpy as np;
import pandas as pandas;
from scipy import ndimage
from torchvision import datasets, transforms, models
import torch.nn.functional as F


import os
#Para corregir error en depuracion usando pycharm, hacer
"""
Look for the Anaconda directory and set the Library\plugins subdir 
(here c:\ProgramData\Anaconda3\Library\plugins) as environment variable 
QT_PLUGIN_PATH
"""
import matplotlib.pyplot as plt
from PIL import Image

archivoPesos = "dogsCats_weights1"

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

  HTTP error 403 while getting http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
  Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
Could not install requirement torch==0.4.1 from http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl because of HTTP error 403 Client Error: Forbidden for url: http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl for URL http://download.pytorch.org/whl/cu100/torch-0.4.1-cp36-cp36m-linux_x86_64.whl


**Creacion de la red y sus  operaciones**

Creamos la clase RedConvolucionalMNIST, la cual hereda de la clase nn.Module de Pytorch.
Crea las siguientes capas y operaciones: 


1.   Capa de convolucion 1: Con una sola capa de entrada (escala de grises), 10 filtros de 5x5 dimensiones
2.   Capa de convolucion 2: 10 capas de entrada, 20 filtros de 5x5 dimensiones
3.   Operacion de dropout: Para regularizar el modelo, se crea la operacion de dropout la   cual descativa unidades de forma aleatoria.
4.   Capa completamente conectada 1: Tiene 320 unidades a la entrada y 50 a la salida.
5.   Capa completamente conectadas 2: Con 50 unidades de entrada y 10 a la salida, correspondiente a las K = 10 clases a clasificar.

El cálculo de la pasada hacia adelante, se hace al apilar las siguientes capas:



1.   [28x28x1] Capa de entrada, para imagenes de MNIST a escala de grises
2.   [24x24x10] Convolucion 1: Aplicacion de 10 filtros de 5x5, a un paso p = 1 y sin zero padding.
3.  [12x12x10] Max pooling 1: Max pooling de 2x2
4.  [8x8x20] Convolucion 2: 20 filtros de 5x5, a un paso p = 1, y sin zero padding
5.  [4x4x20] Max pooling 2: Max pooling de 2x2
6.  [1x320] Aplanado de los datos, 320 = 4x4x20
7.  [1x50]Capa completamente conectada de 320  unidades de entrada y 50 de salida con funcion ReLU de activacion
8.  [1x10]Capa completamente conectada de 50 unidades a 10 con funcion Softmax de activacion







In [0]:
"""
Clase que hereda del paquete neural net de pytorch
"""
class RedConvolucionalGatosPerros(nn.Module):
    """
    Constructor de la clase RecConvolucional
    Crea las capas a evaluar en la pasada hacia adelante o forward
    Cada capa sera un atributo en la clase
    """
    def __init__(self):
        super(RedConvolucionalGatosPerros, self).__init__()
        
        res50_model = models.resnet50(pretrained=True)
        self.res50_conv = nn.Sequential(*list(res50_model.children())[:-2])
        
        for param in self.res50_conv.parameters():
          param.requires_grad = True
        
        
        
        
        #se crea una operacion convolucional con 10 filtros, y 1 un canal a la entrada
        #Cada filtro es de 5x5
        #entrada es de 256x256
        self.capaConvolucion1 = nn.Conv2d(1, 10, kernel_size = 5)
        #(256-5) + 1 = 252x252 a la salida, con 10 mapas de activacion
        #Operacion convolucional con 10 canales a la entrada y 20 filtros
        #Cada filtro es de 5x5
        self.capaConvolucion2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.dropOutConvolucion = nn.Dropout2d()
        
        #crea una transformacion lineal de la forma y = x A^t + b (sin funcion de activacion)
        #A tiene 320 filas y 50 columnas, x es la entrada de 1 fila y 320 columnas
        #Modelo completamente conectado
        
        
        #la salida del promediado global es 2048 promedios de los 2048 feature maps que genera resnet
        self.capaCompletamenteConectada1 = nn.Linear(2048, 2)
        #2 clases, perro o gato
        

    """
    Pasada hacia adelante, sobrecargado de la clase nn neural network
    @param x, muestra a estimar su salida
    """
    def forward(self, x):
        #Apila las capas
        #Convolucion1 -> maxPooling 2x2 (se escoge el maximo en una ventana de 2x2)
        
        
        x =  self.res50_conv(x)
        
        #print("LAYER CONTENT")
     
        
        
        #outputs 2048 activation maps of 8x8 
        #https://resources.wolframcloud.com/NeuralNetRepository/resources/ResNet-50-Trained-on-ImageNet-Competition-Data
        
        
        #print("RESNET 50 OUTPUT")
        #print(xResnet.shape)
        
        
        #print("Dims to do average")
        #print(xResnet.size()[2:])
        
        
        x =  F.avg_pool2d(x, x.size()[2:])
        #x dims torch.Size([15, 2048, 1, 1])
        
        #print("AFTER AVERAGE OUTPUT")
        #print(x.shape)
        
        x = x.view(-1, 2048)
        
        #print("AFTER flattening")
        #print(x.shape)
        
        # Conv1 -> mP -> Conv2 -> DropOutConv2 -> MaxPool -> Relu -> FC1 -> ReLU
        x = F.relu(self.capaCompletamenteConectada1(x))
        # Conv1 -> mP -> Conv2 -> DropOutConv2 -> MaxPool -> Relu -> FC1 -> ReLU -> Dropout
        x = F.dropout(x, training = self.training)
        # Conv1 -> mP -> Conv2 -> DropOutConv2 -> MaxPool -> Relu -> FC1 -> ReLU -> Dropout -> FC2 
        
        output = F.log_softmax(x, dim = 1)
        print("Output dims")
        print(output.shape)
        
        
        return output

# Data set loader
It is recommended to overload the data set loader

In [0]:
import pandas as pd
import numpy as np
from PIL import Image
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data.dataset import Dataset  # For custom datasets

class DatasetCatsDogs(Dataset):
    def __init__(self, samplesDirectory, transform = None):
        """
        Inits the dataset
        :param samplesDirectory: path to the directory with the samples
        :param transform: Pytorch transforms list
        """
        self.transform = transform
        (sampleIds, labels2) = self.fillSampleIdsAndLabels(samplesDirectory)
        self.labels2 = labels2
        self.sampleIds = sampleIds


    def __getitem__(self, index):
        """
        :param index: dataset sample index
        :return: (sample, label)
        """
        samplePath = self.sampleIds[index]
        # Open the image
        imagePIL = Image.open(samplePath)
        #apply transformations
        if self.transform is not None:
            img_as_tensor2 = self.transform(imagePIL)
        #read the label
        y = self.labels2[index]
        #transform label to numpy array
        #y = np.array([y]);
        return (img_as_tensor2, y)

    def __len__(self):
        """
        Returns the length of the dataset
        :return: number of samples
        """
        numSamples = len(self.sampleIds)
        return numSamples


    def fillSampleIdsAndLabels(self, samplesDirectory):
        """
        Extracts sample and label metadata
        :param samplesDirectory: directory of samples
        :return: (file names, label array)
        """
        files = []
        labels = []
        # r=root, d=directories, f = files
        for r, d, f in os.walk(samplesDirectory):
            for file in f:
                if '.jpg' in file:
                    samplePath = os.path.join(r, file)
                    files.append(samplePath)
                    if('cat' not in samplePath):
                        labels += [0.0]
                    else:
                        labels += [1.0]
        return (files, labels)
    
    
def testDataset():
    transformations = transforms.Compose([transforms.transforms.Resize((256, 256)), transforms.ToTensor() ])



    custom_mnist_from_csv = DatasetCatsDogs("/content/drive/My Drive/Parma/LibroVersionMasReciente/4_Clasificacion/RedesNeuronales/SrcRedesConvolucionales/dataCatsDogs/train", transformations)

    
    
    
    mn_dataset_loader = torch.utils.data.DataLoader(dataset=custom_mnist_from_csv, batch_size = 12, shuffle = True)

    for i, (images, labels) in enumerate(mn_dataset_loader):
        images = Variable(images)
        labels = Variable(labels)
        print(images.shape)
        print(labels.shape)
        break
    
testDataset()
    


torch.Size([12, 3, 256, 256])
torch.Size([12])


** Entrenado y prueba de la red**
El proceso de entrenamiento consiste en tomar un lote de muestras aleatorio, de tamaño especifico, calcular la pasada hacia adelante, y luego la perdida. Posteriormente se propaga hacia atrás el error y se actualizan los pesos. 


1.   La función de pérdida es la entropía cruzada

El parámetro de desviacion estandar define la degradación a aplicar en las muestras.
En la función de pruebas se evalúa la exactitud usando el cargador de muestras de prueba





In [0]:
"""
Entrena el modelo recibido
@param args, argumentos del entrenamiento
@param modelo, modelo a entrenar
@param dispositivo, dispositivo a utilizar
@param cargadorEntrenamiento, cargador de datos de entrenamiento
@param optimizador, optimizador a utilizar durante el entrenamiento

"""
def entrenarModelo(args, modelo, dispositivo, cargadorEntrenamiento, optimizador, numEpochs):
    #se le envia el mensaje al modelo de que iniciara su entrenamiento (para por ejemplo activar dropout)
    modelo.train()
    for idLote, (loteMuestrasEntrada, loteSalidasEsperadas) in enumerate(cargadorEntrenamiento):

        
        loteSalidasEsperadas = loteSalidasEsperadas.long()
        # aloca la muestra  y la etiqueta en el dispositivo deseado
        loteMuestrasEntrada, loteSalidasEsperadas = loteMuestrasEntrada.to(dispositivo), loteSalidasEsperadas.to(dispositivo)


        #Los gradientes se ponen en 0
        optimizador.zero_grad()
        #Se estima la salida del modelo con la muestra de entrada


       

        loteSalidasEstimadas = modelo(loteMuestrasEntrada)
        perdida = F.cross_entropy(loteSalidasEstimadas, loteSalidasEsperadas)
        
        print("GRAD MODELO")
        list(modelo.parameters())[0].grad 
        
        
        # Computa los gradientes
        #perdida.backward()
        # Actualiza los gradientes
        #optimizador.step()
        
        
        
        a = list(modelo.parameters())[0].clone()
        perdida.backward()
        optimizador.step()
        b = list(modelo.parameters())[0].clone()
        print("Prueba debe dar false")
        print(torch.equal(a.data, b.data))
        
        
        
        
        #cada tantos epochs se imprime el error
        if idLote % args.log_interval == 0:
            print('Epoch de entrenamiento: {} [{}/{} ({:.0f}%)]\tPerdida: {:.6f}'.format(
                numEpochs, idLote * len(loteMuestrasEntrada), len(cargadorEntrenamiento.dataset),
                           100. * idLote / len(cargadorEntrenamiento), perdida.item()))
            #guarda el modelo
            torch.save(modelo.state_dict(), archivoPesos)
            
            
"""
Prueba el modelo
@param modelo, modelo a probar
@param dispositivo, dispositivo a utilizar
@param cargadorMuestrasPrueba, carga las muestras de prueba a utilizar

"""
def probarModelo(modelo, dispositivo, cargardorMuestrasPrueba):
    #el modelo sera evaluado, se le envia tal mensaje
    modelo.eval()
    perdidaPruebas = 0
    muestrasCorrectas = 0
    #se previene a pytorch de no realizar el calculo del gradiente
    with torch.no_grad():
        #por cada par u, t
        for loteMuestrasEntrada, loteSalidasEspera in cargardorMuestrasPrueba:
            
            #se aloca la muestra en el dispositivo especificado
            loteMuestrasEntrada, loteSalidasEspera = loteMuestrasEntrada.to(dispositivo), loteSalidasEspera.to(dispositivo)

            
            
            #calculo de la salida estimada
            loteSalidasEstimadas = modelo(loteMuestrasEntrada)
            #calculo de la perdida de las pruebas, usando la entropia cruzada
            perdidaPruebas += F.cross_entropy(loteSalidasEstimadas, loteSalidasEspera).item() # sum up batch loss
            #Toma el indice con la mayor probabilidad
            etiquetaPredicha = loteSalidasEstimadas.max(1, keepdim = True)[1]
            #numero de muestras correctamente predichas
            muestrasCorrectas += etiquetaPredicha.eq(loteSalidasEspera.view_as(etiquetaPredicha)).sum().item()
    #perdida promedio
    perdidaPruebas /= len(cargardorMuestrasPrueba.dataset)
    print('\nConjunto de pruebas: Perdida promedio: {:.4f}, Precision: {}/{} ({:.0f}%)\n'.format(
        perdidaPruebas, muestrasCorrectas, len(cargardorMuestrasPrueba.dataset),
        100. * muestrasCorrectas / len(cargardorMuestrasPrueba.dataset)))
    return 100. * muestrasCorrectas / len(cargardorMuestrasPrueba.dataset);


** Funciones de carga de datos y ejecucion del modelo **

Se utilizará el conjunto de datos de MNIST. Pytorch permite la transformación de los datos a cargar, en este caso los mismos se pasan a tensor y se normalizan con la media y desviación estándar conocidas del conjunto de datos.


In [0]:



"""
Carga los datos sin transformacion de ruido
@param args, argumentos de cargga de datos, como el tamanio del lote
@param kwargs, rutas de los datos
@return cargadorMuestrasEntrenamiento y cargadorMuestrasPrueba
"""
def cargarDatos(args, kwargs):
     
    #Normalize the dataset and transform to grayscale   
    transformations = transforms.Compose([transforms.transforms.Resize((256, 256)), transforms.ToTensor(), transforms.Normalize([0.485], [0.229]) ])
        
        
    dataSetTraining = DatasetCatsDogs("/content/drive/My Drive/Parma/LibroVersionMasReciente/4_Clasificacion/RedesNeuronales/SrcRedesConvolucionales/dataCatsDogs/train", transformations)
    
    dataSetValidation = DatasetCatsDogs("/content/drive/My Drive/Parma/LibroVersionMasReciente/4_Clasificacion/RedesNeuronales/SrcRedesConvolucionales/dataCatsDogs/train", transformations)
    
    cargadorMuestrasEntrenamiento = torch.utils.data.DataLoader(dataset=dataSetTraining, batch_size = 30, shuffle = True)
    
    cargadorMuestrasPrueba = torch.utils.data.DataLoader(dataset=dataSetValidation, batch_size = 30, shuffle = True)

    return (cargadorMuestrasEntrenamiento, cargadorMuestrasPrueba);


"""
Ejecuta el modelo en un dispositivo, con los datos cargados y argumentos definidos
@param dispositivo, dispositivo en el que se ejecutra la optimizacion y prueba del modelo
@param args, argumentos del modelo
@param cargadorEntrenamiento, cargador de las muestras de entrenamiento
@param cargadorPruebas, cargador de las muestras para la puesta a prueba del modelo
@return perdidaModelo, retorna la perdida total del modelo
"""
def ejecutarModelo(dispositivo, args, cargadorEntrenamiento, cargadorPruebas):
    # se pasa el modelo al dispositivo indicado
    modelo = RedConvolucionalGatosPerros().to(dispositivo)
    # se tratan de cargar los pesos
    loadWeights = args.loadWeights;
    if (loadWeights):
        try:
            modelo.load_state_dict(torch.load(archivoPesos))
        except:
            print("No se pudieron cargar los pesos")
    # se define el optimizador
    optimizer = optim.SGD(modelo.parameters(), lr=args.lr, momentum=args.momentum)
    # se entrena y prueba el modelo segun el numero de epochs
    for epoch in range(1, args.epochs + 1):
        entrenarModelo(args, modelo, dispositivo, cargadorEntrenamiento, optimizer, epoch)
        perdidaModelo = probarModelo(modelo, dispositivo, cargadorPruebas);
    return perdidaModelo;


** Almacenamiento del modelo y carga de parametros **

Las siguientes funciones gestionan la escritura de resultados, y la lectura de parametros, usando Pandas y el Parser de Python, respectivamente.





In [0]:
"""
Crea el data frame necesario
@param numCorridas, numero de corridas total a ejecutar, para reportar resultados estadisticamente significaivo
@return marcoDatos, retorna el marco de datos con tal formato
"""
def crearContenedorResultados(numCorridas):
    datos = {'Datos_ruido_std': numCorridas * [0],
                'Datos_sin_ruido': numCorridas * [0]}
    marcoDatos = pandas.DataFrame(datos, columns=['Datos_ruido_std', 'Datos_sin_ruido'])
    #ejemplo de escritura al frame
    marcoDatos.set_value(0, 'Datos_ruido_std', 500)
    marcoDatos.to_csv("Prueba1.csv");
    return marcoDatos;
  
"""
Crea el convertidor o parser de los argumentos
@return parser
"""

def crearParserArgumentos():
    # Configuracion del entrenamiento
    parser = argparse.ArgumentParser(description = 'PyTorch MNIST Example')
    # Tamanio del lote de entrenamiento
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
    # Tamanio del lote de pruebas
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    # Cantidad de epochs o iteraciones
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    # Coeficiente de aprendizaje
    parser.add_argument('--lr', type=float, default=1000, metavar='LR',
                        help='learning rate (default: 0.001)')
    # Momentum
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    # CUDA
    parser.add_argument('--no-cuda', action='store_true', default=True,
                        help='disables CUDA training')
    # Semilla?
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    # Intervalo en volcar a la bitacora
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')

    # Carga de pesos
    parser.add_argument('--loadWeights', type=int, default=False, metavar='W',
                        help='Load the weights from file')

    args = parser.parse_args(args=[])
    return args;

** Ejecucion del Modelo **
Funcion principal del programa


In [0]:

"""
Funcion principal

VER ESTE!
"""
def main():
    args = crearParserArgumentos();
    #define si se usara CUDA
    usarCUDA = torch.cuda.is_available()
    print("Using CUDA?")
    print(usarCUDA)
    
    #se define la semilla
    torch.manual_seed(args.seed)
    #dispositivo
    dispositivo = torch.device("cuda" if usarCUDA else "cpu")
    #cantidad de obreros para carga de datos
    kwargs = {'num_workers': 1, 'pin_memory': True} if usarCUDA else {}
    numCorridas = 5;
    
    marcoDatos = crearContenedorResultados(numCorridas);


    (cargadorEntrenamiento, cargadorPruebas) = cargarDatos(args, kwargs);
    #genera los datos para con y sin ruido
    for i in range(0, numCorridas):
        print("------CORRIDA: ", i);
        perdidaModeloRuido = ejecutarModelo(dispositivo, args, cargadorEntrenamiento, cargadorPruebas);
        
        


if __name__ == '__main__':
    main()

Using CUDA?
True


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()


------CORRIDA:  0
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Epoch de entrenamiento: 1 [0/25000 (0%)]	Perdida: 0.693414
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
False
Epoch de entrenamiento: 1 [300/25000 (1%)]	Perdida: 0.693147
Output dims
torch.Size([30, 2])
GRAD MODELO
Prueba debe dar false
Fals

KeyboardInterrupt: ignored

Authors: *Saul Calderon